<a href="https://colab.research.google.com/github/wisrovi/RedesNeuronales/blob/master/AprendizajePorRefuerzo-PongDeterministic2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Teoría

## Objetivo: 
Repasar los conceptos vistos en clase.

La puntuación de este bloque es de 4 puntos sobre la nota final.

**Define brevemente qué es el aprendizaje por refuerzo. ¿Qué diferencias hay entre aprendizaje supervisado, no supervisado y por refuerzo?**

**Define con tus palabras los conceptos de Entorno, Agente, Recompensa, Estado y Observación.**

**Dependiendo del algoritmo de aprendizaje por refuerzo que se use, ¿qué clasificaciones podemos encontrar? Coméntalas brevemente.**




**Lista tres diferencias entre los algoritmos de DQN y Policy Gradient**

# Práctica

Objetivo: Implementar una solución, usando keras-rl y basada en el algoritmo de DQN visto en clase, para que un agente aprenda una estrategia ganadora en el juego del Pong.

La puntuación de este bloque es de 6 puntos sobre la nota final.

El entorno sobre el que trabajaremos será _PongDeterministic-v0_ y el algoritmo que usaremos será _DQN_.

Para evaluar cómo lo está haciendo el agente, la recompensa en el _Pong_ oscila, aproximadamente, en el rango de valores **[-20, 20]**. La estrategia óptima de un agente estaría alrededor de una media de recompensa de 20.

- **NOTA IMPORTANTE**: Si el agente no llegara a aprender una estrategia ganadora, responder sobre la mejor estrategia obtenida.

# Drive

In [27]:
#https://medium.com/@shivamrawat_756/how-to-prevent-google-colab-from-disconnecting-717b88a128c0
from IPython.display import display, HTML
js = ('<script>function ConnectButton(){ '
        'console.log("Connect pushed"); '
        'document.querySelector("#connect").click()} '
        'setInterval(ConnectButton,3000);</script>')
display(HTML(js))

In [0]:
isColabGoogle = True
enlazarGoogleDrive = False
entrenar = True

In [0]:
BASE_FOLDER = ""
if enlazarGoogleDrive:
    from google.colab import drive
    drive.mount('/content/gdrive')
    RUTA_GOOGLE = '/content/gdrive/My Drive/Master IA/AprendizajePorRefuerzo/'

In [4]:
!ls -la '/content/gdrive/My Drive/Master IA/AprendizajePorRefuerzo/'

ls: cannot access '/content/gdrive/My Drive/Master IA/AprendizajePorRefuerzo/': No such file or directory


# Install and import

### Install

In [0]:
!pip install keras-rl==0.4.2

#!pip install tensorflow==1.13.1
!pip install tensorflow-gpu==1.14

!pip install gym
!pip install gym[atari]
!pip install h5py


#creo son necesarias, pero no se ha comprobado que en su ausencia falle
!pip install jupyter
!pip install torch

ERROR: Operation cancelled by user


### Import

In [0]:
from PIL import Image

import numpy as np
import gym

# Para las librerias para la red neuronal
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from keras.optimizers import Adam, SGD, RMSprop

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor

from rl.callbacks import FileLogger, ModelIntervalCheckpoint
from keras.callbacks import TensorBoard
import os 

### Init

In [3]:
ENV_NAME = 'PongDeterministic-v0'

# Get the environment and extract the number of actions.
env = gym.make(ENV_NAME)
nb_actions = env.action_space.n

# random seed
np.random.seed(123)
env.seed(123)

[123, 151010689]

In [0]:
# Define the input shape to resize the screen
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

# This processor will be similar to the Atari processor
class PongProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        
        img = Image.fromarray(observation)
        # resize and convert to grayscale
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')  # saves storage in experience memory

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

In [0]:
processor = PongProcessor()

# CNN

In [0]:
def createModel(transferModel = None):
    input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE

    model = Sequential()
    if transferModel is not None:
        model.add(transferModel)
    else:
        model.add(Permute((2, 3, 1), input_shape=input_shape))

    model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(nb_actions))
    model.add(Activation('sigmoid'))  
    return model


In [7]:
model = createModel()
print(model.summary())




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 32)        8224      
_________________________________________________________________
activation_1 (Activation)    (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
activation_2 (Activation)    (None, 9, 9, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
activation_3 (Activation)    (None, 7, 7, 64)      

# Memory and policy

In [0]:
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)

policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05,
                              nb_steps=1000000)

#policy = BoltzmannQPolicy(tau=1.)

# Agente

## Create

In [0]:
dqn = DQNAgent(
    nb_actions=nb_actions, 
    model=model, 
    policy=policy, 
    memory=memory,
    processor=processor, 
    nb_steps_warmup=10000, 
    target_model_update=1e-2,
    gamma=.99)


## Optimizer

In [0]:
optimizer = Adam(lr=.00025)
optimizer = SGD()
optimizer = RMSprop (lr = .00025, rho = 0.95, epsilon = 0.01)

## Compile

In [11]:
#compilar modelo
dqn.compile(optimizer, metrics=['mae'])

In [0]:
def GuardarPesos(nombreArchivo):
    dqn.save_weights(BASE_FOLDER + nombreArchivo, overwrite=True)
    if enlazarGoogleDrive:
        dqn.save_weights(RUTA_GOOGLE + nombreArchivo, overwrite=True)   
    print("Pesos guardados") 

def TestGeneral(eposidiosEvaluar = 10, debugTest = False):
    print()
    acumuladorReward = 0
    acumuladorSteps = 0

    print("Ejecutando test", end="")
    for i in range(eposidiosEvaluar):
        print(".", end="")
        verLog = 0
        if debugTest:
            print("*******************************************************")
            print("*                        %d                             *" %(i))
            print("*******************************************************")
            verLog = 1

        rta = dqn.test(env, 
                      nb_episodes=1, 
                      visualize=graficosVisualesGame,
                      verbose=verLog).history
        reward = rta['episode_reward'][0]
        steps = rta['nb_steps'][0]

        if debugTest:
            print("episode_reward: ",reward)
            print("nb_steps: ",steps)
        
        acumuladorReward += reward
        acumuladorSteps += steps
    print()
    promedioSteps = acumuladorSteps / eposidiosEvaluar
    promedioReward = acumuladorReward / eposidiosEvaluar

    print("Promedio Reward: ",  promedioReward)
    print("Promedio Steps: ",  promedioSteps)

## callbacks

In [0]:
callbacksParametrizados = list()

In [0]:
#Para algunos casos es importante saber cual entrenamiento fue mejor, 
#este callback guarda el modelo tras cada epoca completada con el fin de si luego se desea un registro de pesos para cada epoca
#Se ha usado este callback para poder optener el mejor modelo de pesos, sobretodo en la red neuronal creada desde cero
#siendo de gran utilidad para determinar el como ir modificando los layer hasta obtener el mejor modelo
checkpoint_weights_filename = 'dqn_' + ENV_NAME + '_weights_{step}.h5f'
guardarModeloCadaCiertoIntervaloDeSteps = ModelIntervalCheckpoint(checkpoint_weights_filename, interval=50000)

callbacksParametrizados.append(guardarModeloCadaCiertoIntervaloDeSteps)

In [0]:
log_filename = 'dqn_{}_log.json'.format(ENV_NAME)
guardarModelo = FileLogger(log_filename, interval=100)

callbacksParametrizados.append(guardarModelo)

In [0]:
tb_log_dir = 'logs/tmp'
tb_callback = TensorBoard(log_dir=tb_log_dir, histogram_freq=0, write_graph=True)

callbacksParametrizados.append(tb_callback)

In [17]:
"""
#EarlyStopping, detener el entrenamiento una vez que su pérdida comienza a aumentar 
early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=5, #argumento de patience representa el número de épocas antes de detenerse una vez que su pérdida comienza a aumentar (deja de mejorar). 
    min_delta=0,  #es un umbral para cuantificar una pérdida en alguna época como mejora o no. Si la diferencia de pérdida es inferior a min_delta , se cuantifica como no mejora. Es mejor dejarlo como 0 ya que estamos interesados ​​en cuando la pérdida empeora.
    mode='auto')

callbacksParametrizados.append(early_stop)
"""

"\n#EarlyStopping, detener el entrenamiento una vez que su pérdida comienza a aumentar \nearly_stop = EarlyStopping(\n    monitor='val_loss', \n    patience=5, #argumento de patience representa el número de épocas antes de detenerse una vez que su pérdida comienza a aumentar (deja de mejorar). \n    min_delta=0,  #es un umbral para cuantificar una pérdida en alguna época como mejora o no. Si la diferencia de pérdida es inferior a min_delta , se cuantifica como no mejora. Es mejor dejarlo como 0 ya que estamos interesados \u200b\u200ben cuando la pérdida empeora.\n    mode='auto')\n\ncallbacksParametrizados.append(early_stop)\n"

In [18]:
"""
#ReduceLROnPlateau, que si el entrenamiento no mejora tras unos epochs específicos, reduce el valor de learning rate del modelo
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.1, 
    patience=5, 
    epsilon=1e-4, 
    mode='min')

callbacksParametrizados.append(reduce_lr)
"""

"\n#ReduceLROnPlateau, que si el entrenamiento no mejora tras unos epochs específicos, reduce el valor de learning rate del modelo\nreduce_lr = ReduceLROnPlateau(\n    monitor='val_loss', \n    factor=0.1, \n    patience=5, \n    epsilon=1e-4, \n    mode='min')\n\ncallbacksParametrizados.append(reduce_lr)\n"

In [19]:
len(callbacksParametrizados)

3

# Fit

## Config

In [0]:
usarPesosEntrenados = True
testearAlFinalEntrenamiento = False  #si se hace un testeo, se guarda solo si hay mejoria
guardarSiempreDespuesCadaEntrenamiento = True
graficosVisualesGame = False

In [0]:
totalSteps = 500000
intervalo = 5000

cantidadEntrenamientos = 50 #cantidad de veces que repito el entrenamiento con el totalSteps

#si totalSteps = 100000 y intervalo = 10000, entonces se haran 10 intervalos

In [0]:
#archivo guardar pesos
weights_filename = 'dqn_' + ENV_NAME + '_weights3.h5f'

## Load old training

In [23]:
if isColabGoogle:
    from google.colab import files

    uploaded = files.upload()

    for fn in uploaded.keys():
      print('User uploaded file "{name}" with length {length} bytes'.format(
          name=fn, length=len(uploaded[fn])))

NameError: ignored

In [28]:
primerTest = True
if usarPesosEntrenados:    
    if isColabGoogle:
        print("Archivos disponibles: ")
        !ls '/content/gdrive/My Drive/Master IA/AprendizajePorRefuerzo/'
        print("Buscando archivo: " + weights_filename)
    try:              
        dqn.load_weights(BASE_FOLDER + weights_filename)
        print("Carga de archivo de pesos completa.")
        if primerTest:
            #scoreOld = dqn.test(env, nb_episodes=5, visualize=graficosVisualesGame).history['episode_reward'][1]
            #TestGeneral(10, True)
            dqn.test(env, 
                nb_episodes=10, 
                visualize=graficosVisualesGame)
    except:
        scoreOld = -50
        print("No se encontro archivos de pesos")
        pass

Archivos disponibles: 
ls: cannot access '/content/gdrive/My Drive/Master IA/AprendizajePorRefuerzo/': No such file or directory
Buscando archivo: dqn_PongDeterministic-v0_weights3.h5f
No se encontro archivos de pesos


## True Training

In [29]:
GuardarPesos(weights_filename)

Pesos guardados


In [30]:
if entrenar:    
    try:
        dqn.fit(env, 
                callbacks=callbacksParametrizados, 
                nb_steps=31000, 
                log_interval=intervalo, 
                visualize=graficosVisualesGame) #, 
                #verbose=2)
    except:
        pass
    if primerTest:        
        TestGeneral(5)
    GuardarPesos(weights_filename)



Training for 31000 steps ...
Interval 1 (0 steps performed)
5000/5000 [==============================] - 18s 4ms/step - reward: -0.0224
5 episodes - episode_reward: -20.600 [-21.000, -20.000] - ale.lives: 0.000

Interval 2 (5000 steps performed)
5000/5000 [==============================] - 17s 3ms/step - reward: -0.0224
5 episodes - episode_reward: -20.400 [-21.000, -19.000] - ale.lives: 0.000

Interval 3 (10000 steps performed)
5000/5000 [==============================] - 141s 28ms/step - reward: -0.0212
6 episodes - episode_reward: -20.667 [-21.000, -20.000] - loss: 0.012 - mean_absolute_error: 0.180 - mean_q: 0.215 - mean_eps: 0.989 - ale.lives: 0.000

Interval 4 (15000 steps performed)
5000/5000 [==============================] - 140s 28ms/step - reward: -0.0226
5 episodes - episode_reward: -20.200 [-21.000, -20.000] - loss: 0.012 - mean_absolute_error: 0.005 - mean_q: 0.001 - mean_eps: 0.984 - ale.lives: 0.000

Interval 5 (20000 steps performed)
5000/5000 [======================

In [0]:
if entrenar:
    conteo = 0
    entrenar = False
    agenteOld = None
    for i in range(cantidadEntrenamientos):
        print("*****************************************************")
        print("*****************************************************")
        print("**                                                 **")
        print("**                                                 **")
        print("**                                                 **")
        print("**                       %d                         **" %(i))
        print("**                                                 **")
        print("**                                                 **")
        print("**                                                 **")
        print("*****************************************************")
        print("*****************************************************")
        # Training part
        try:
            dqn.fit(env, 
                    callbacks=callbacksParametrizados, 
                    nb_steps=totalSteps, 
                    log_interval=intervalo, 
                    visualize=graficosVisualesGame) #, 
                    #verbose=2)            
        except:
            print("Error al entrenar, se ha interrumpido el fit")
            pass
        
        if guardarSiempreDespuesCadaEntrenamiento:
            GuardarPesos(weights_filename)
       
        if testearAlFinalEntrenamiento:
            testEval = dqn.test(env, nb_episodes=5, visualize=graficosVisualesGame).history['episode_reward'][1]       

            if scoreOld < testEval:
                scoreOld = testEval
                agenteOld = dqn
                GuardarPesos("Avanced-" + weights_filename)
                print("Guardando pesos", scoreOld)
            else:
                print("No hubo mejora, no se guarda este modelo.")
                conteo += 1
                if conteo > 3:
                    pass
                    break
                dqn = agenteOld   
        else:
            GuardarPesos(weights_filename)
            #dqn.test(env, nb_episodes=5, visualize=graficosVisualesGame)
            TestGeneral(5)
else:    
    dqn.load_weights(weights_filename)

*****************************************************
*****************************************************
**                                                 **
**                                                 **
**                                                 **
**                       0                         **
**                                                 **
**                                                 **
**                                                 **
*****************************************************
*****************************************************
Training for 500000 steps ...
Interval 1 (0 steps performed)
5000/5000 [==============================] - 17s 3ms/step - reward: -0.0226
5 episodes - episode_reward: -20.400 [-21.000, -20.000] - ale.lives: 0.000

Interval 2 (5000 steps performed)
5000/5000 [==============================] - 17s 3ms/step - reward: -0.0214
5 episodes - episode_reward: -20.200 [-21.000, -20.000] - ale.lives: 0.000

Interva

In [0]:
GuardarPesos(weights_filename)

## Test

In [0]:
TestGeneral(15)